# MPIA Arxiv on Deck 2: Debugging notebook

In this notebook, I keep some first order commands for diagnostic of issues with papers.
Main definitions are taken from the main notebook.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

# Some useful definitions.
class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

We get the author list from the MPIA website

In [2]:
# Getting the list of authors can take sometimes (internet connection)
# Caching the MPIA author list to avoid running this line every time we restart the kernel.
import yaml
try:
    with open('tmp_mpia_authors.yml', 'r') as fin:
        mpia_authors = yaml.load(fin, yaml.BaseLoader)
    print("`mpia.get_mpia_mitarbeiter_list()`: restored from cache")
except FileNotFoundError:
    print("`mpia.get_mpia_mitarbeiter_list()`: cannot be restored from cache.")
    # get list from MPIA website
    # it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
    mpia_authors = mpia.get_mpia_mitarbeiter_list()
    with open('tmp_mpia_authors.yml', 'w') as fout:
        fout.write(yaml.dump(mpia_authors))

`mpia.get_mpia_mitarbeiter_list()`: restored from cache


In [3]:
# Get family names only
data = [k[-1] for k in mpia_authors]

# clean -- Having titles is something new at MPIA
data = [k.replace('Dr. ', '').strip() for k in data]

filtered_data = list(filter(mpia.filter_non_scientists, data))

name_variations = filter(lambda x: x is not None,
                         [mpia.consider_variations(name) for name in filtered_data])
mitarbeiter_list = sorted(filtered_data + list(name_variations))

lst = [(mpia.get_special_corrections(mpia.get_initials(name)), name) for name in mitarbeiter_list]
lst = [(mpia.family_name_from_initials(k[0]), k[0], k[1]) for k in lst]
lst = sorted(lst, key=lambda x: x[0])
lst = set(lst)

We get the paper to debug

In [4]:
which = "2212.09168"
paper = get_paper_from_identifier(which)
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2212.09168-b31b1b.svg)](https://arxiv.org/abs/2212.09168) | **PHANGS-JWST First Results: Rapid Evolution of Star Formation in the Central Molecular Gas Ring of NGC1365**  |
|| Eva Schinnerer, et al. |
|*Appeared on*| *2022-12-18*|
|*Comments*| *24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| Large-scale bars can fuel galaxy centers with molecular gas, often leading tothe development of dense ring-like structures where intense star formationoccurs, forming a very different environment compared to galactic disks. Wepair ~0.3" (30pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1)mapping of the central ~5kpc of the nearby barred spiral galaxy NGC1365, toinvestigate the physical mechanisms responsible for this extreme starformation. The molecular gas morphology is resolved into two well-known brightbar lanes that surround a smooth dynamically cold gas disk (R_gal ~ 475pc)reminiscent of non-star-forming disks in early type galaxies and likely fed bygas inflow triggered by stellar feedback in the lanes. The lanes host a largenumber of JWST-identified massive young star clusters. We find some evidencefor temporal star formation evolution along the ring. The complex kinematics inthe gas lanes reveal strong streaming motions and may be consistent withconvergence of gas streamlines expected there. Indeed, the extreme line-widthsare found to be the result of inter-`cloud' motion between gas peaks; ScousePydecomposition reveals multiple components with line widths of <sigma_CO,scouse>~ 19km/s and surface densities of <Sigma_H2,scouse> ~ 800M_sun/pc^2, similar tothe properties observed throughout the rest of the central molecular gasstructure. Tailored hydro-dynamical simulations exhibit many of the observedproperties and imply that the observed structures are transient and highlytime-variable. From our study of NGC1365, we conclude that it is predominantlythe high gas inflow triggered by the bar that is setting the star formation inits CMZ.|

In [5]:
# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in lst]

hl_authors = highlight_authors_in_list(paper['authors'], hl_list, verbose=True)
matches = [(hl, orig) for hl, orig in zip(hl_authors, paper['authors']) if 'mark' in hl]
if not matches:
    warnings.warn(AffiliationWarning("WARNING: This paper does not seem to have MPIA authors."))
paper['authors'] = hl_authors
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2212.09168-b31b1b.svg)](https://arxiv.org/abs/2212.09168) | **PHANGS-JWST First Results: Rapid Evolution of Star Formation in the Central Molecular Gas Ring of NGC1365**  |
|| <mark>Eva Schinnerer</mark>, et al. -- incl., <mark>Jonathan D. Henshaw</mark>, <mark>Stephen Hannon</mark>, <mark>Justus Neumann</mark>, <mark>Sophia K. Stuber</mark> |
|*Appeared on*| *2022-12-18*|
|*Comments*| *24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue*|
|**Abstract**| Large-scale bars can fuel galaxy centers with molecular gas, often leading tothe development of dense ring-like structures where intense star formationoccurs, forming a very different environment compared to galactic disks. Wepair ~0.3" (30pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1)mapping of the central ~5kpc of the nearby barred spiral galaxy NGC1365, toinvestigate the physical mechanisms responsible for this extreme starformation. The molecular gas morphology is resolved into two well-known brightbar lanes that surround a smooth dynamically cold gas disk (R_gal ~ 475pc)reminiscent of non-star-forming disks in early type galaxies and likely fed bygas inflow triggered by stellar feedback in the lanes. The lanes host a largenumber of JWST-identified massive young star clusters. We find some evidencefor temporal star formation evolution along the ring. The complex kinematics inthe gas lanes reveal strong streaming motions and may be consistent withconvergence of gas streamlines expected there. Indeed, the extreme line-widthsare found to be the result of inter-`cloud' motion between gas peaks; ScousePydecomposition reveals multiple components with line widths of <sigma_CO,scouse>~ 19km/s and surface densities of <Sigma_H2,scouse> ~ 800M_sun/pc^2, similar tothe properties observed throughout the rest of the central molecular gasstructure. Tailored hydro-dynamical simulations exhibit many of the observedproperties and imply that the observed structures are transient and highlytime-variable. From our study of NGC1365, we conclude that it is predominantlythe high gas inflow triggered by the bar that is setting the star formation inits CMZ.|

We get the (TeX) source
* retrieve the tarball
* find the main tex file and parse it
* parse for affiliations (but debugging so we do not stop if not found)
* generate the the output markdown

In [6]:
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)

# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
doc.highlight_authors_in_list(hl_list, verbose=True)

full_md = doc.generate_markdown_text()

# replace citations
try:
    bibdata = latex_bib.LatexBib.from_doc(doc)
    full_md = latex_bib.replace_citations(full_md, bibdata)
except Exception as e:
    raise e

/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex.py:359: LatexWarning: Latex injecting: 'ngc1365_center_astro-ph.bbl' from 'tmp_2212.09168/ngc1365_center_astro-ph.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Found 129 bibliographic references in tmp_2212.09168/ngc1365_center_astro-ph.bbl.


/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{a}})}]{LEROY2_PHANGSJWST}{Leroy}, A., {et~al.} subm.{\natexlab{a}}, \apjl
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{b}})}]{LEROY1_PHANGSJWST}---. subm.{\natexlab{b}}, \apjl
 regex = 
        \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
        
  warnings.warn(str(e))
/Users/fouesneau/Work/arxiv_display/venv/lib/python3.11/site-packages/arxiv_on_deck_2/latex_bib.py:132: UserWarning: Error processing bibitem
 item =

In [7]:
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\comm}[1]{{\color{red} \textbf{#1}}}$
$\newcommand{\Msun}{\mathrm{M}_{\sun}}$
$\newcommand{\fr}[1]{{\color{green} \textbf{#1}}}$</div>



<div id="title">

# PHANGS-JWST First Results: Rapid Evolution of Star Formation in the Central Molecular Gas Ring of NGC 1365

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2212.09168-b31b1b.svg)](https://arxiv.org/abs/2212.09168) _24 pages, 8 figures, accepted for publications as part of PHANGS-JWST ApJL Focus Issue_

</div>
<div id="authors">

<mark><mark>Eva Schinnerer</mark></mark>, et al. -- incl., <mark><mark>Jonathan D. Henshaw</mark></mark>, <mark><mark>Stephen Hannon</mark></mark>, <mark><mark>Justus Neumann</mark></mark>, <mark><mark>Sophia K. Stuber</mark></mark>

</div>
<div id="abstract">

**Abstract:** Large-scale bars can fuel galaxy centers with molecular gas, often leading to the development of dense ring-like structures where intense star formation occurs, forming a very different environment compared to galactic disks. We pair $\sim$ 0.3 $\arcsec$ (30 pc) resolution new JWST/MIRI imaging with archival ALMA CO(2-1) mapping of the central $\sim$ 5 kpc of the nearby barred spiral galaxy NGC 1365, to investigate the physical mechanisms responsible for this extreme star formation.The molecular gas morphology is resolved into two well-known bright bar lanes that surround a smooth dynamically cold gas disk ( $\rm R_{gal} \sim 475 pc$ ) reminiscent of non-star-forming disks in early type galaxies and likely fed by gas inflow triggered by stellar feedback in the lanes. The lanes host a large number of JWST-identified massive young star clusters. We find some evidence for temporal star formation evolution along the ring.The complex kinematics in the gas lanes reveal strong streaming motions and may be consistent with convergence of gas streamlines expected there. Indeed, the extreme line-widths are found to be the result of inter-`cloud' motion between gas peaks; ${\sc ScousePy}$ decomposition reveals multiple components with line widths of $\rm \langle\sigma_{CO,scouse}\rangle \approx 19 km s^{-1}$ and surface densities of $\rm \langle \Sigma_{H_2,scouse}\rangle \approx  800 M_{\odot} pc^{-2}$ , similar to the properties observed throughout the rest of the central molecular gas structure.  Tailored hydro-dynamical simulations exhibit many of the observed properties and imply that the observed structures are transient and highly time-variable.  From our study of NGC 1365, we conclude that it is predominantly the high gas inflow triggered by the bar that is setting the star formation in its CMZ.

</div>

<div id="div_fig1">

<img src="tmp_2212.09168/./figures/scouse2x2.png" alt="Fig4" width="100%"/>

**Figure 4. -** {\sc ScousePy} decomposition of the 30 pc resolution molecular CO(2-1) gas emission. Multiple Gaussian components are needed to fit $\sim31\%$ of the data. This is demonstrated most clearly in the spiral arms, where two and three component models are common (up to six components in the southwestern arm; _top left_). Spectra from selected regions with the resulting {\sc ScousePy} fits typically exhibit well separated line components (_top right_). The inferred peak CO temperature and velocity dispersions of the individual {\sc ScousePy} components (_bottom left_) occupy a similar region of parameter space independent of whether they are derived from single or multiple components fits (color-coding). The grey shaded regions here correspond to the limits of our fitting (a signal-to-noise ratio of 2 and a single channel width).
    The three-dimensional representation of the individual {\sc ScousePy} components located within the white dashed box in the top left panel, displayed in $ppv$ space (_bottom right_), reveals an inner smoothly rotating disk (highlighted in black; extracted from the ellipse in the top left panel) while the bar lanes show strong local fluctuations in velocity indicating an increasing complexity of the kinematics in the molecular gas there (the color here refers to the peak CO temperature of the individual components). The physical spacing of the $pp$ grid in this image is $\sim475$ pc. (*fig:scouse*)

</div>
<div id="div_fig2">

<img src="tmp_2212.09168/./figures/scouse2x2.png" alt="Fig12" width="100%"/>

**Figure 12. -** {\sc ScousePy} decomposition of the 30 pc resolution molecular CO(2-1) gas emission. Multiple Gaussian components are needed to fit $\sim31\%$ of the data. This is demonstrated most clearly in the spiral arms, where two and three component models are common (up to six components in the southwestern arm; _top left_). Spectra from selected regions with the resulting {\sc ScousePy} fits typically exhibit well separated line components (_top right_). The inferred peak CO temperature and velocity dispersions of the individual {\sc ScousePy} components (_bottom left_) occupy a similar region of parameter space independent of whether they are derived from single or multiple components fits (color-coding). The grey shaded regions here correspond to the limits of our fitting (a signal-to-noise ratio of 2 and a single channel width).
    The three-dimensional representation of the individual {\sc ScousePy} components located within the white dashed box in the top left panel, displayed in $ppv$ space (_bottom right_), reveals an inner smoothly rotating disk (highlighted in black; extracted from the ellipse in the top left panel) while the bar lanes show strong local fluctuations in velocity indicating an increasing complexity of the kinematics in the molecular gas there (the color here refers to the peak CO temperature of the individual components). The physical spacing of the $pp$ grid in this image is $\sim475$ pc. (*fig:scouse*)

</div>
<div id="div_fig3">

<img src="tmp_2212.09168/./figures/MIR_images_v5.png" alt="Fig3" width="100%"/>

**Figure 3. -** Distribution of the emission from PAHs and hot dust emission as probed by JWST/MIRI observations at $\sim$0.3$\arcsec$ resolution, which is comparable to the resolution of the ALMA CO(2-1) data (North is up, East to the left.).
    _Left:_ The F770W filter with a $\sim$0.24$\arcsec$ resolution reveals the overall distribution of the neutral ISM as its emission is dominated by the  7.7$\mu$m  PAH feature. For reference the distribution of the CO emission is shown by a single blue contour. Note that the central position is significantly affected by saturation due to the bright AGN and dominated by its PSF causing the artefacts. Also the three brightest compact sources north of the AGN are saturated.
    _Right:_ Several embedded young star-forming clusters are evident in the 10 $\mu$m emission and stand out as bright compact sources. Our 10 $\mu$m-selected sources are highlighted by the cyan diamond symbols and are only found in the region shown. About 57\% of these objects coincide with the young ($<$10 Myr) star clusters studied by \citet[][shown as green crosses]{WHITMORE_PHANGSJWST} and have estimated stellar masses of $\rm 10^6  M_{\odot}$ or higher. The 21 $\mu$m-selected sources from  ([Hassani, Rosolowsky and Leroy (2022)]())  that are classified as ISM emitting sources are shown as cyan circles. The dearth of clusters in the bar lane southwest of the nucleus is notable. (*fig:jwst*)

</div>

In [47]:
import re
from typing import Sequence
from arxiv_on_deck_2.latex_bib import clean_special_characters, BibliographyData, LatexBib


def extract_bibitem_info(item_str: str) -> dict:
    """Get groups of info from bibitem definition
    Thanks to https://regex101.com/
    """
    regex_href = r"""
    \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3}).*?href(.*?{(?P<url>http[\S]*)})(?P<rest>.*)
    """
    regex_nohref = r"""
    \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
    """

    # You can manually specify the number of replacements by changing the 4th argument
    regex = regex_href if r'\href' in item_str else regex_nohref

    # replace special characters
    item_str = item_str.replace(r'\&', '')    
    item_str = clean_special_characters(item_str)
    matches = re.search(regex, item_str.replace('\n', ' '),
                        re.DOTALL | re.IGNORECASE | re.VERBOSE | re.MULTILINE)
    try:
        info = matches.groupdict()
    except AttributeError as e:
        raise RuntimeError(f"Error processing bibitem\n item = {item_str}\n regex = {regex}")
    info.setdefault('title', "")
    info.setdefault('url', "")
    return info

def find_bibitem(sample: str) -> Sequence[str]:
    """ Find bibitem definitions in a string"""
    entries = re.findall(r'(\\bibitem)((.(?!bibitem))*)',
                         sample, re.DOTALL | re.IGNORECASE | re.VERBOSE | re.MULTILINE)
    entries = [''.join(entry).replace('\n', '').strip() for entry in entries]
    return entries

def clean_author(authors):
    """ Put things in the right format for bibtex """
    regex = r"(?P<last>{[\w{}\~\-\\]+})[,\s]+?(?P<first>[\w{}\~\-\\\.\s]+),*"
    return ['{0:s}, {1:s}'.format(*it)
            for it in re.findall(regex, authors.strip())]

def get_bibtex_code(rk: dict):
    tpl = """
@article{{{bibkey:s},
title = "{title:s}",
author = {{{authors:s}}},
url = "{url:s}",
year = "{year:s}"
}}
"""
    rk['authors'] = ' and '.join(clean_author(rk['authors']))
    return tpl.format(**rk)

In [48]:
fname = "tmp_2212.09168/ngc1365_center_astro-ph.bbl"
# Read file
with open(fname, 'r') as fin:
    content = fin.read()

# identify entries
entries = find_bibitem(content)
n_entries = len(entries)
print(f"Found {n_entries:,d} bibliographic references in {fname:s}.")

# extract individual fields per entry
r = []
for it in entries:
    try:
        r.append(extract_bibitem_info(it))
    except RuntimeError as e:
        warnings.warn(str(e))

## create the bibtex text
bibtex = ''.join(get_bibtex_code(rk) for rk in r)
db = LatexBib(BibliographyData.from_string(bibtex, 'bibtex'))

Found 129 bibliographic references in tmp_2212.09168/ngc1365_center_astro-ph.bbl.


/var/folders/jb/twt_8kfx4mj86m9v8jgccs_r5gbl99/T/ipykernel_4859/923640648.py:17: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{a}})}]{LEROY2_PHANGSJWST}{Leroy}, A., {et~al.} subm.{\natexlab{a}}, \apjl
 regex = 
    \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
    
  warnings.warn(str(e))
/var/folders/jb/twt_8kfx4mj86m9v8jgccs_r5gbl99/T/ipykernel_4859/923640648.py:17: UserWarning: Error processing bibitem
 item = \bibitem[{{Leroy} {et~al.}(subm.{\natexlab{b}})}]{LEROY1_PHANGSJWST}---. subm.{\natexlab{b}}, \apjl
 regex = 
    \\bibitem(\[[^\[\]]*?\]){(?P<bibkey>[a-zA-Z0-9\-\+\.\S]+?)}(?P<authors>|([\D]*?))(?P<year>[12][0-9]{3})(?P<rest>.*)
    
  warnings.warn(str(e))
/var/folders/jb/twt_8kfx4mj86m9v8jgccs_r5gbl99/T/ipykernel_4859/923640648.py:17: UserWarning: Error processing bibitem
 item = \bibitem[{{Liu} {et~al.}(subm.)}]{LIU_PHANGSJWST}{Liu}, D., {et~al.} subm., \apjl
 r

In [50]:
db.bibdata

BibliographyData(
  entries=OrderedCaseInsensitiveDict([
    ('agertz+2015', Entry('article',
      fields=[
        ('title', ''), 
        ('url', ''), 
        ('year', '2015')],
      persons=OrderedCaseInsensitiveDict([('author', [Person('{Agertz}, O.'), Person('{Kravtsov}, A. V.')])]))), 
    ('agertz+2021', Entry('article',
      fields=[
        ('title', ''), 
        ('url', ''), 
        ('year', '2021')],
      persons=OrderedCaseInsensitiveDict([('author', [Person('{Agertz}, O.'), Person('{Renaud}, F.'), Person('{Feltzing}, S.')])]))), 
    ('Alonso-Herrero2012', Entry('article',
      fields=[
        ('title', ''), 
        ('url', ''), 
        ('year', '2012')],
      persons=OrderedCaseInsensitiveDict([('author', [Person('{Alonso-Herrero}, A.'), Person('{S{á}nchez-Portal}, M.')])]))), 
    ('alonso-herrero2020', Entry('article',
      fields=[
        ('title', ''), 
        ('url', ''), 
        ('year', '2020')],
      persons=OrderedCaseInsensitiveDict([('author', 

In [41]:
!cat tmp_2212.09168/ngc1365_center_astro-ph.bbl

\begin{thebibliography}{}
\expandafter\ifx\csname natexlab\endcsname\relax\def\natexlab#1{#1}\fi
\providecommand{\url}[1]{\href{#1}{#1}}
\providecommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}
\providecommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}
\providecommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}

\bibitem[{{Agertz} \& {Kravtsov}(2015)}]{agertz+2015}
{Agertz}, O., \& {Kravtsov}, A.~V. 2015, \apj, 804, 18,
  \dodoi{10.1088/0004-637X/804/1/18}

\bibitem[{{Agertz} {et~al.}(2021){Agertz}, {Renaud}, {Feltzing}, {Read},
  {Ryde}, {Andersson}, {Rey}, {Bensby}, \& {Feuillet}}]{agertz+2021}
{Agertz}, O., {Renaud}, F., {Feltzing}, S., {et~al.} 2021, \mnras, 503, 5826,
  \dodoi{10.1093/mnras/stab322}

\bibitem[{{Alonso-Herrero} {et~al.}(2012){Alonso-Herrero},
  {S{\'a}nchez-Portal}, {Ramos Almeida}, {Pereira-Santaella}, {Esquej},
  {Garc{\'\i}a-Burillo}, {Castillo}, {Gonz{\'a}lez-Mart{\'\i}n}, {L

In [7]:
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\afk}[1]{\textcolor{purple}{\uline{#1}}}$
$\newcommand{\afks}[1]{\textcolor{purple}{\sout{#1}}}$
$\newcommand{\il}[1]{\textcolor{cyan}{\uline{#1}}}$
$\newcommand{\ils}[1]{\textcolor{cyan}{\sout{#1}}}$
$\newcommand{\wf}[1]{\textcolor{blue}{\uline{#1}}}$
$\newcommand{\wfs}[1]{\textcolor{blue}{\sout{#1}}}$</div>



<div id="title">

# The initial mass function and other stellar properties across the core of the Hydra I cluster$\footnote{This paper includes data gathered with the 6.5 meterMagellan Telescopes located at Las Campanas Observatory, Chile.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2303.00044-b31b1b.svg)](https://arxiv.org/abs/2303.00044) _32 pages, 18 figures. Accepted for publication in ApJ_

</div>
<div id="authors">

Ilaria Lonoce, W. L. Freedman, <mark>A. Feldmeier-Krause</mark>

</div>
<div id="abstract">

**Abstract:** The Hydra I cluster offers an excellent opportunity to study and compare the relic old stellar populations in the core of its two brightest galaxies. In addition, the differing kinematics of the two galaxies allows a test of the local validity of general scaling relations. In this work we present a direct comparison employing full spectral fitting of new high-quality long-slit optical and NIR spectroscopic  data. We retrieve age, metallicity and 19 elemental abundances out to $\sim12$ kpc within each galaxy, as well as the IMF in their central regions.Our results suggest that the inner $\sim5$ kpc region of both galaxies, despite their different masses, formed at the same time and evolved with a similar star formation time-scale and chemical enrichment, confirming their early formation in the cluster build up. Only the overall metallicity and IMF radial profiles show differences connected with their different velocity dispersion profiles. The radial trend of the IMF positively correlates with both [ Z/H ] and $\sigma$ . While the trends of the IMF with metallicity agree with a global trend for both galaxies, the trends with the velocity dispersion exhibit differences. The outer regions show signs of mixed stellar populations with large differences in chemical content compared to the centers, but with similar old ages.

</div>

<div id="div_fig1">

<img src="tmp_2303.00044/./figures/ALF_ageZHimfALPHA_Hydra_FINAL.png" alt="Fig8" width="100%"/>

**Figure 8. -** \small{Age, metallicity and IMF slope trends across the Hydra I cluster center as retrieved with ALF. West is on the left, East is on the right. The bottom panel
shows the derived mismatch parameter $\alpha_r$.
Dash-dotted horizontal tan lines in the IMF and $\alpha_r$ panels outline the corresponding values for a Salpeter IMF, dashed those for a Kroupa IMF. Open diamonds refer to regions where the results are less robust due to lower S/N, velocity dispersion broadening and/or lack of important features. } (*fig:ageZimf*)

</div>
<div id="div_fig2">

<img src="tmp_2303.00044/./figures/Literature_comparison_KIN_PAPER.png" alt="Fig11" width="100%"/>

**Figure 11. -** \small{Velocity dispersion (upper panel) and radial velocity (lower panel) profiles as retrieved with ALF (orange).
Green triangles and red circles show an indication of the trends retrieved by the works of  ([Richtler, Salinas and Misgeld (2011)]())  and  ([Hilker, Richtler and Barbosa (2018)]()) , respectively, as they appear in figure $6$ of  ([Hilker, Richtler and Barbosa (2018)]())  for the case of position angle $108$◦.  Typical error bars are shown on the right of both panels.
} (*fig:kinematics*)

</div>
<div id="div_fig3">

<img src="tmp_2303.00044/./figures/ALF_ALLelements_Hydra_moreMask_FINALproc.png" alt="Fig9" width="100%"/>

**Figure 9. -** Similar to Figure \ref{fig:ageZimf but for all the retrieved elemental abundances.} (*fig:elements*)

</div>

In [8]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [9]:
export_markdown_summary(full_md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2212.09168.md
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/scouse2x2.png
    + _build/html/tmp_2212.09168/./figures/MIR_images_v5.png
